In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/__script__.py
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/plotly-6.1.2-py3-none-any.whl
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/lightgbm-4.6.0-py3-none-manylinux_2_28_x86_64.whl
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/contourpy-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/fonttools-4.58.4-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/catboost-1.2.8-cp311-cp311-manylinux2014_x86_64.whl
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl
/kaggle/input/pm-85615954-at-06-22-2025-18-37-24/pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64

In [2]:
# !pip install rdkit lightgbm catboost xgboost

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
import time
import warnings
warnings.filterwarnings('ignore')

In [4]:
print(f"Starting enhanced polymer property prediction pipeline...")
print(f"Date: 2025-06-22")
print(f"User: Koshik Debanath")
start_time = time.time()

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)

# Load data
train = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
submission = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

# Check for missing values
print("\nMissing values in train data:")
missing_values = train.isnull().sum()
print(missing_values)

# Calculate available samples for each property
target_cols = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
available_samples = {col: train.shape[0] - missing_values[col] for col in target_cols}
print("\nAvailable samples for each property:")
for col, count in available_samples.items():
    print(f"{col}: {count} ({count/train.shape[0]*100:.2f}%)")

# Target property statistics
print("\nTarget properties statistics:")
print(train[target_cols].describe())

# Calculate property ranges (needed for wMAE calculation)
property_ranges = {}
for col in target_cols:
    property_ranges[col] = train[col].dropna().max() - train[col].dropna().min()
    
print("\nEstimated property ranges:")
for col, range_val in property_ranges.items():
    print(f"{col}: {range_val:.4f}")

# Calculate weights for wMAE metric
weights = {}
for col in target_cols:
    # Weight = (1/sqrt(n_i)) / range_i, normalized
    weights[col] = (1 / np.sqrt(available_samples[col])) / property_ranges[col]

# Normalize weights to sum to number of tasks (5)
weight_sum = sum(weights.values())
for col in target_cols:
    weights[col] = weights[col] / weight_sum * len(target_cols)

print("\nEstimated weights for wMAE metric:")
for col, weight in weights.items():
    print(f"{col}: {weight:.4f}")

# Define RDKit descriptors calculator once
descriptor_names = [x[0] for x in Descriptors._descList]
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

# Safe version of polymer-specific features
def get_safe_polymer_features(mol):
    """Extract basic polymer-specific features without using problematic functions."""
    features = {}
    
    # Basic counts
    features['num_atoms'] = mol.GetNumAtoms()
    features['num_heavy_atoms'] = mol.GetNumHeavyAtoms()
    features['num_bonds'] = mol.GetNumBonds()
    
    # Count atom types
    atom_types = {'C': 0, 'N': 0, 'O': 0, 'S': 0, 'F': 0, 'Cl': 0, 'Br': 0, 'I': 0}
    aromatic_atoms = 0
    
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        if symbol in atom_types:
            atom_types[symbol] += 1
        
        # Count aromatic atoms
        if atom.GetIsAromatic():
            aromatic_atoms += 1
    
    features['aromatic_atoms'] = aromatic_atoms
    
    # Add atom type counts to features
    for atom_type, count in atom_types.items():
        features[f'num_{atom_type}'] = count
        
        # Calculate ratios if there are heavy atoms
        if mol.GetNumHeavyAtoms() > 0:
            features[f'ratio_{atom_type}'] = count / mol.GetNumHeavyAtoms()
        else:
            features[f'ratio_{atom_type}'] = 0
    
    # Count bond types
    bond_types = {Chem.rdchem.BondType.SINGLE: 0, 
                  Chem.rdchem.BondType.DOUBLE: 0, 
                  Chem.rdchem.BondType.TRIPLE: 0,
                  Chem.rdchem.BondType.AROMATIC: 0}
    
    for bond in mol.GetBonds():
        bond_type = bond.GetBondType()
        if bond_type in bond_types:
            bond_types[bond_type] += 1
    
    # Add bond type counts to features
    features['num_single_bonds'] = bond_types[Chem.rdchem.BondType.SINGLE]
    features['num_double_bonds'] = bond_types[Chem.rdchem.BondType.DOUBLE]
    features['num_triple_bonds'] = bond_types[Chem.rdchem.BondType.TRIPLE]
    features['num_aromatic_bonds'] = bond_types[Chem.rdchem.BondType.AROMATIC]
    
    # Calculate property descriptors that are reliable
    try:
        features['mw'] = Descriptors.MolWt(mol)
        features['logp'] = Descriptors.MolLogP(mol)
        features['tpsa'] = Descriptors.TPSA(mol)
        features['num_rotatable_bonds'] = Descriptors.NumRotatableBonds(mol)
        features['num_h_donors'] = Descriptors.NumHDonors(mol)
        features['num_h_acceptors'] = Descriptors.NumHAcceptors(mol)
        features['num_rings'] = Descriptors.RingCount(mol)
        features['num_aromatic_rings'] = Descriptors.NumAromaticRings(mol)
        features['num_aliphatic_rings'] = Descriptors.NumAliphaticRings(mol)
    except:
        # If any descriptor fails, set it to 0
        for desc in ['mw', 'logp', 'tpsa', 'num_rotatable_bonds', 'num_h_donors', 
                     'num_h_acceptors', 'num_rings', 'num_aromatic_rings', 'num_aliphatic_rings']:
            if desc not in features:
                features[desc] = 0
    
    # Custom polymer-relevant ratios
    if mol.GetNumHeavyAtoms() > 0:
        features['rotatable_per_heavy'] = features['num_rotatable_bonds'] / mol.GetNumHeavyAtoms()
        features['rings_per_heavy'] = features.get('num_rings', 0) / mol.GetNumHeavyAtoms()
        features['aromatic_atom_ratio'] = features.get('aromatic_atoms', 0) / mol.GetNumHeavyAtoms()
    else:
        features['rotatable_per_heavy'] = 0
        features['rings_per_heavy'] = 0
        features['aromatic_atom_ratio'] = 0
    
    return features

# Improved feature engineering from SMILES - FIXED COLUMN ISSUE
def generate_molecule_features(smiles_list):
    """Generate robust molecular features from SMILES strings with fixed column handling."""
    # First, create a sample molecule to get the exact feature structure
    sample_mol = Chem.MolFromSmiles('CC')
    sample_descriptors = list(calculator.CalcDescriptors(sample_mol))
    sample_polymer_features = get_safe_polymer_features(sample_mol)
    
    # Calculate sample fingerprints
    sample_morgan_fp = AllChem.GetMorganFingerprintAsBitVect(sample_mol, 2, nBits=256)
    sample_morgan_features = np.zeros((256,))
    AllChem.DataStructs.ConvertToNumpyArray(sample_morgan_fp, sample_morgan_features)
    
    sample_maccs_fp = MACCSkeys.GenMACCSKeys(sample_mol)
    sample_maccs_features = np.zeros((167,))
    AllChem.DataStructs.ConvertToNumpyArray(sample_maccs_fp, sample_maccs_features)
    
    # Create feature names based on the sample
    descriptor_names = [x[0] for x in Descriptors._descList]
    polymer_feature_names = list(sample_polymer_features.keys())
    morgan_feature_names = [f'morgan_{i}' for i in range(len(sample_morgan_features))]
    maccs_feature_names = [f'maccs_{i}' for i in range(len(sample_maccs_features))]
    
    # Combine all feature names
    feature_names = descriptor_names + polymer_feature_names + morgan_feature_names + maccs_feature_names
    
    # Create empty feature lists
    features = []
    valid_indices = []
    
    for i, smiles in enumerate(smiles_list):
        try:
            # Try various SMILES parsing approaches
            mol = None
            
            # 1. Try direct parsing
            mol = Chem.MolFromSmiles(smiles)
            
            # 2. If that fails, try with the assumption this might be a polymer SMILES notation
            if mol is None and '*' in smiles:
                modified_smiles = smiles.replace('*', 'C')
                mol = Chem.MolFromSmiles(modified_smiles)
            
            # 3. If still fails, try sanitizing with parameters
            if mol is None:
                mol = Chem.MolFromSmiles(smiles, sanitize=False)
                if mol is not None:
                    try:
                        Chem.SanitizeMol(mol, sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL^Chem.SanitizeFlags.SANITIZE_KEKULIZE)
                    except:
                        pass
            
            if mol is not None:
                # Calculate RDKit descriptors
                descriptors = list(calculator.CalcDescriptors(mol))
                
                # Calculate polymer-specific features
                polymer_features = list(get_safe_polymer_features(mol).values())
                
                # Calculate fingerprints
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    # Morgan fingerprints
                    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)
                    morgan_features = np.zeros((256,))
                    AllChem.DataStructs.ConvertToNumpyArray(morgan_fp, morgan_features)
                    
                    # MACCS keys
                    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
                    maccs_features = np.zeros((167,))
                    AllChem.DataStructs.ConvertToNumpyArray(maccs_fp, maccs_features)
                
                # Combine all features
                all_features = descriptors + polymer_features + list(morgan_features) + list(maccs_features)
                
                # Ensure we have the right number of features
                if len(all_features) != len(feature_names):
                    print(f"Warning: Feature length mismatch for SMILES {smiles}: {len(all_features)} features vs {len(feature_names)} names")
                    # Pad or truncate to match feature_names length
                    if len(all_features) < len(feature_names):
                        all_features = all_features + [0] * (len(feature_names) - len(all_features))
                    else:
                        all_features = all_features[:len(feature_names)]
                
                features.append(all_features)
                valid_indices.append(i)
            else:
                if i < 5:  # Print first few failures for debugging
                    print(f"Warning: Could not parse SMILES: {smiles}")
        except Exception as e:
            if i < 5:  # Print first few failures for debugging
                print(f"Error processing SMILES {smiles}: {str(e)}")
    
    print(f"Successfully processed {len(valid_indices)} out of {len(smiles_list)} SMILES strings")
    print(f"Feature vector length: {len(feature_names)}")
    
    # Double-check that all feature vectors have the same length as feature_names
    for i, feat in enumerate(features):
        if len(feat) != len(feature_names):
            features[i] = feat[:len(feature_names)] if len(feat) > len(feature_names) else feat + [0] * (len(feature_names) - len(feat))
    
    return pd.DataFrame(features, index=valid_indices, columns=feature_names), valid_indices

print("\nGenerating enhanced molecular features from SMILES...")
train_features, train_valid_idx = generate_molecule_features(train['SMILES'])
test_features, test_valid_idx = generate_molecule_features(test['SMILES'])

# Join features with original data
train_with_features = pd.concat([
    train.iloc[train_valid_idx].reset_index(drop=True),
    train_features.reset_index(drop=True)
], axis=1)

test_with_features = pd.concat([
    test.iloc[test_valid_idx].reset_index(drop=True),
    test_features.reset_index(drop=True)
], axis=1)

print(f"Train with features shape: {train_with_features.shape}")
print(f"Test with features shape: {test_with_features.shape}")

# Clean features - remove inf, nan, and extreme values
def clean_features(df, feature_cols):
    """Clean features by removing inf values and extreme outliers."""
    df_clean = df.copy()
    
    # Replace infinity with NaN
    df_clean[feature_cols] = df_clean[feature_cols].replace([np.inf, -np.inf], np.nan)
    
    # Check for any columns with all NaN or constant values
    valid_cols = []
    for col in feature_cols:
        if df_clean[col].notna().sum() > 0 and df_clean[col].nunique() > 1:
            valid_cols.append(col)
    
    print(f"Retained {len(valid_cols)} out of {len(feature_cols)} features after removing invalid columns")
    
    # Detect and cap extreme outliers (beyond 5 std deviations)
    for col in valid_cols:
        if df_clean[col].dtype.kind in 'fc':  # Only numerical columns
            mean = df_clean[col].mean()
            std = df_clean[col].std()
            if std > 0:
                lower_bound = mean - 5 * std
                upper_bound = mean + 5 * std
                df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)
    
    return df_clean, valid_cols

# Get all possible feature columns
all_feature_cols = train_features.columns.tolist()
print(f"Total number of features: {len(all_feature_cols)}")

train_with_features, valid_feature_cols = clean_features(train_with_features, all_feature_cols)
test_with_features, _ = clean_features(test_with_features, all_feature_cols)

# Feature selection for each target
def select_features_for_target(df, target_col, all_features, max_features=500):
    """Select most important features for a specific target."""
    # Filter to rows with target value
    df_valid = df[df[target_col].notna()].copy()
    
    if len(df_valid) < 50:  # Not enough data
        return all_features[:min(len(all_features), max_features)]
    
    # Check correlation with target
    correlations = []
    for col in all_features:
        if df_valid[col].dtype.kind in 'fc':  # Only numerical columns
            corr = df_valid[col].corr(df_valid[target_col])
            if not pd.isna(corr):
                correlations.append((col, abs(corr)))
    
    # Sort by correlation and take top features
    correlations.sort(key=lambda x: x[1], reverse=True)
    top_features = [x[0] for x in correlations[:max_features]]
    
    print(f"Selected {len(top_features)} features for {target_col} based on correlation")
    return top_features

# Prepare target-specific feature sets
target_features = {}
for col in target_cols:
    target_features[col] = select_features_for_target(train_with_features, col, valid_feature_cols)

# Get optimized hyperparameters for each property
def get_property_hyperparams(property_name):
    """Get optimized hyperparameters for each property."""
    # These are pre-optimized hyperparameters for each property
    params = {
        'Tg': {
            'xgb': {
                'n_estimators': 1000,
                'learning_rate': 0.01,
                'max_depth': 6,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'min_child_weight': 3,
                'reg_alpha': 0.01,
                'reg_lambda': 1.0
            },
            'cat': {
                'iterations': 1000,
                'learning_rate': 0.03,
                'depth': 6,
                'l2_leaf_reg': 3
            }
        },
        'FFV': {
            'xgb': {
                'n_estimators': 2000,
                'learning_rate': 0.005,
                'max_depth': 8,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'min_child_weight': 2,
                'reg_alpha': 0.1,
                'reg_lambda': 0.5
            },
            'cat': {
                'iterations': 1500,
                'learning_rate': 0.02,
                'depth': 7,
                'l2_leaf_reg': 2
            }
        },
        'Tc': {
            'xgb': {
                'n_estimators': 1500,
                'learning_rate': 0.01,
                'max_depth': 7,
                'subsample': 0.85,
                'colsample_bytree': 0.75,
                'min_child_weight': 3,
                'reg_alpha': 0.05,
                'reg_lambda': 1.0
            },
            'cat': {
                'iterations': 1200,
                'learning_rate': 0.02,
                'depth': 5,
                'l2_leaf_reg': 4
            }
        },
        'Density': {
            'xgb': {
                'n_estimators': 1200,
                'learning_rate': 0.01,
                'max_depth': 6,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'min_child_weight': 2,
                'reg_alpha': 0.1,
                'reg_lambda': 0.5
            },
            'cat': {
                'iterations': 1000,
                'learning_rate': 0.03,
                'depth': 6,
                'l2_leaf_reg': 3
            }
        },
        'Rg': {
            'xgb': {
                'n_estimators': 1000,
                'learning_rate': 0.02,
                'max_depth': 7,
                'subsample': 0.8,
                'colsample_bytree': 0.7,
                'min_child_weight': 3,
                'reg_alpha': 0.05,
                'reg_lambda': 1.0
            },
            'cat': {
                'iterations': 1200,
                'learning_rate': 0.02,
                'depth': 7,
                'l2_leaf_reg': 3
            }
        }
    }
    
    return params.get(property_name, params['Tg'])  # Default to Tg if property not found

# Create a class for ensemble prediction
class AveragingEnsemble:
    def __init__(self, models, imputer, scaler):
        self.models = models
        self.imputer = imputer
        self.scaler = scaler
        
    def predict(self, X):
        try:
            X = X.copy()
            X = X.replace([np.inf, -np.inf], np.nan)
            X_imputed = self.imputer.transform(X)
            X_imputed = np.nan_to_num(X_imputed, nan=0.0, posinf=0.0, neginf=0.0)
            X_scaled = self.scaler.transform(X_imputed)
            
            preds = np.column_stack([model.predict(X_scaled) for model in self.models])
            return np.mean(preds, axis=1)
        except Exception as e:
            print(f"Prediction error: {str(e)}")
            return np.zeros(X.shape[0])

# Mean predictor for fallback
class MeanPredictor:
    def __init__(self, value):
        self.value = value
        
    def predict(self, X):
        return np.full(len(X), self.value)

# Advanced model training with ensemble
def train_advanced_model(df, target_col, feature_cols, n_splits=5):
    """Train advanced ensemble models for a specific target property."""
    # Filter valid rows for this target
    valid_idx = df[target_col].notna()
    df_valid = df.loc[valid_idx]
    
    # Check if we have enough data
    if len(df_valid) < 30:
        print(f"Not enough data for {target_col}, using mean value prediction")
        mean_val = df_valid[target_col].mean() if len(df_valid) > 0 else 0
        return {'model': MeanPredictor(mean_val), 'cv_score': 0.0}
    
    # Prepare the data
    X = df_valid[feature_cols].copy()
    y = df_valid[target_col].values
    
    # Handle missing values with KNN imputation for better accuracy
    imputer = KNNImputer(n_neighbors=5)
    X_imputed = imputer.fit_transform(X)
    
    # Transform features for better model performance
    scaler = PowerTransformer(method='yeo-johnson')
    X_scaled = scaler.fit_transform(X_imputed)
    X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0)
    
    print(f"Training advanced models on {len(y)} samples for {target_col}")
    
    # Set up cross-validation
    n_splits = min(n_splits, len(y) // 10 or 2)  # Ensure we have enough samples per fold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    # Get optimized hyperparameters for this property
    hyperparams = get_property_hyperparams(target_col)
    
    # Set up base models
    all_models = []
    oof_preds = np.zeros(len(y))
    cv_scores = []
    
    # Train and cross-validate
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        fold_models = []
        fold_preds = []
        
        # 1. XGBoost model
        try:
            xgb_model = xgb.XGBRegressor(
                **hyperparams['xgb'],
                random_state=SEED+fold
            )
            xgb_model.fit(X_train, y_train)
            xgb_preds = xgb_model.predict(X_val)
            fold_preds.append(xgb_preds)
            fold_models.append(xgb_model)
            print(f"  Fold {fold+1} XGB MAE: {mean_absolute_error(y_val, xgb_preds):.6f}")
        except Exception as e:
            print(f"  XGB model failed: {str(e)}")
        
        # 2. CatBoost model
        try:
            cat_model = CatBoostRegressor(
                **hyperparams['cat'],
                random_seed=SEED+fold,
                verbose=False
            )
            cat_model.fit(X_train, y_train)
            cat_preds = cat_model.predict(X_val)
            fold_preds.append(cat_preds)
            fold_models.append(cat_model)
            print(f"  Fold {fold+1} CatBoost MAE: {mean_absolute_error(y_val, cat_preds):.6f}")
        except Exception as e:
            print(f"  CatBoost model failed: {str(e)}")
        
        # 3. Random Forest model
        try:
            rf_model = RandomForestRegressor(
                n_estimators=200,
                max_depth=12,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=SEED+fold,
                n_jobs=-1
            )
            rf_model.fit(X_train, y_train)
            rf_preds = rf_model.predict(X_val)
            fold_preds.append(rf_preds)
            fold_models.append(rf_model)
            print(f"  Fold {fold+1} RF MAE: {mean_absolute_error(y_val, rf_preds):.6f}")
        except Exception as e:
            print(f"  RF model failed: {str(e)}")
        
        # 4. Gradient Boosting model
        try:
            gb_model = GradientBoostingRegressor(
                n_estimators=200,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                random_state=SEED+fold
            )
            gb_model.fit(X_train, y_train)
            gb_preds = gb_model.predict(X_val)
            fold_preds.append(gb_preds)
            fold_models.append(gb_model)
            print(f"  Fold {fold+1} GB MAE: {mean_absolute_error(y_val, gb_preds):.6f}")
        except Exception as e:
            print(f"  GB model failed: {str(e)}")
        
        # 5. KNN model for small datasets
        if len(y_train) < 1000:
            try:
                knn_model = KNeighborsRegressor(n_neighbors=7)
                knn_model.fit(X_train, y_train)
                knn_preds = knn_model.predict(X_val)
                fold_preds.append(knn_preds)
                fold_models.append(knn_model)
                print(f"  Fold {fold+1} KNN MAE: {mean_absolute_error(y_val, knn_preds):.6f}")
            except Exception as e:
                print(f"  KNN model failed: {str(e)}")
        
        # Store successful models
        all_models.extend(fold_models)
        
        # Calculate ensemble prediction
        if fold_preds:
            ensemble_preds = np.mean(np.column_stack(fold_preds), axis=1)
            oof_preds[val_idx] = ensemble_preds
            fold_score = mean_absolute_error(y_val, ensemble_preds)
            cv_scores.append(fold_score)
            print(f"  Fold {fold+1} Ensemble MAE: {fold_score:.6f}")
    
    # If all models failed, use a mean predictor
    if not all_models:
        print("  All models failed. Using mean prediction.")
        mean_val = np.mean(y)
        return {
            'model': MeanPredictor(mean_val),
            'cv_score': 0.0
        }
    
    # Calculate overall CV score
    if cv_scores:
        cv_score = mean_absolute_error(y[~np.isnan(oof_preds)], oof_preds[~np.isnan(oof_preds)])
        print(f"Cross-validation MAE for {target_col}: {cv_score:.6f}")
    else:
        cv_score = 0.0
    
    # Create final ensemble model
    final_model = AveragingEnsemble(all_models, imputer, scaler)
    
    return {
        'model': final_model,
        'cv_score': cv_score
    }

# Train advanced models for each target
models = {}
for col in target_cols:
    print(f"\nTraining advanced models for {col}...")
    models[col] = train_advanced_model(
        train_with_features, col, target_features[col]
    )

# Generate test predictions
test_preds = {}
for col in target_cols:
    print(f"Generating predictions for {col}...")
    try:
        test_preds[col] = models[col]['model'].predict(test_with_features[target_features[col]])
    except Exception as e:
        print(f"Error generating predictions for {col}: {str(e)}")
        # Fallback to default values
        fallbacks = {
            'Tg': 400,
            'FFV': 0.2,
            'Tc': 0.2,
            'Density': 1.0,
            'Rg': 10.0
        }
        test_preds[col] = np.full(len(test_with_features), fallbacks[col])

# Create submission dataframe
submission = pd.DataFrame({'id': test_with_features['id']})
for col in target_cols:
    submission[col] = test_preds[col]

# Check submission format
print("\nSubmission preview:")
print(submission.head())

# Ensure all rows from the original test set are included
if len(submission) < len(test):
    print(f"WARNING: Submission has {len(submission)} rows but test set has {len(test)} rows")
    # Fill in missing rows with default values
    missing_ids = set(test['id']) - set(submission['id'])
    print(f"Adding {len(missing_ids)} missing rows")
    
    fallbacks = {
        'Tg': 400,
        'FFV': 0.2,
        'Tc': 0.2,
        'Density': 1.0,
        'Rg': 10.0
    }
    
    for missing_id in missing_ids:
        row = {'id': missing_id}
        row.update(fallbacks)
        submission = pd.concat([submission, pd.DataFrame([row])], ignore_index=True)

# Save submission file
submission.to_csv('submission.csv', index=False)
print("\nSubmission file saved.")

# Calculate approximate wMAE based on CV results
weighted_scores = []
for col in target_cols:
    if 'cv_score' in models[col] and models[col]['cv_score'] > 0:
        weighted_score = models[col]['cv_score'] * weights[col] / property_ranges[col]
        weighted_scores.append(weighted_score)
        print(f"Weighted score for {col}: {weighted_score:.6f}")

if weighted_scores:
    estimated_wmae = sum(weighted_scores)
    print(f"\nEstimated weighted MAE: {estimated_wmae:.6f}")
else:
    print("\nCannot estimate weighted MAE (no CV scores available)")

elapsed_time = time.time() - start_time
print(f"Total runtime: {elapsed_time/60:.2f} minutes")

Starting enhanced polymer property prediction pipeline...
Date: 2025-06-22
User: Koshik Debanath
Train shape: (7973, 7)
Test shape: (3, 2)

Missing values in train data:
id            0
SMILES        0
Tg         7462
FFV         943
Tc         7236
Density    7360
Rg         7359
dtype: int64

Available samples for each property:
Tg: 511 (6.41%)
FFV: 7030 (88.17%)
Tc: 737 (9.24%)
Density: 613 (7.69%)
Rg: 614 (7.70%)

Target properties statistics:
               Tg          FFV          Tc     Density          Rg
count  511.000000  7030.000000  737.000000  613.000000  614.000000
mean    96.452314     0.367212    0.256334    0.985484   16.419787
std    111.228279     0.029609    0.089538    0.146189    4.608640
min   -148.029738     0.226992    0.046500    0.748691    9.728355
25%     13.674509     0.349549    0.186000    0.890243   12.540328
50%     74.040183     0.364264    0.236000    0.948193   15.052194
75%    161.147595     0.380790    0.330500    1.062096   20.411067
max    472.2

[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerator
[18:38:53] DEPRECATION WARNING: please use MorganGenerat

Successfully processed 7973 out of 7973 SMILES strings
Feature vector length: 676


[18:42:11] DEPRECATION WARNING: please use MorganGenerator
[18:42:11] DEPRECATION WARNING: please use MorganGenerator
[18:42:11] DEPRECATION WARNING: please use MorganGenerator
[18:42:11] DEPRECATION WARNING: please use MorganGenerator


Successfully processed 3 out of 3 SMILES strings
Feature vector length: 676
Train with features shape: (7973, 683)
Test with features shape: (3, 678)
Total number of features: 676
Retained 642 out of 676 features after removing invalid columns
Retained 247 out of 676 features after removing invalid columns
Selected 500 features for Tg based on correlation
Selected 500 features for FFV based on correlation
Selected 500 features for Tc based on correlation
Selected 500 features for Density based on correlation
Selected 500 features for Rg based on correlation

Training advanced models for Tg...
Training advanced models on 511 samples for Tg
  Fold 1 XGB MAE: 52.244037
  Fold 1 CatBoost MAE: 52.180999
  Fold 1 RF MAE: 52.296757
  Fold 1 GB MAE: 53.052862
  Fold 1 KNN MAE: 51.691499
  Fold 1 Ensemble MAE: 51.226729
  Fold 2 XGB MAE: 57.571199
  Fold 2 CatBoost MAE: 59.524422
  Fold 2 RF MAE: 57.332819
  Fold 2 GB MAE: 57.248520
  Fold 2 KNN MAE: 62.142682
  Fold 2 Ensemble MAE: 57.840034
 